In [1]:
import pandas as pd
from ecif import *
from os import listdir
import pickle

### Calculating descriptors for some protein-ligand complexes

In [2]:
# Define lists with protein, ligands and names
Ligands = ["Example_Structures/"+x for x in listdir("Example_Structures/") if x[-3:] == "sdf"]
Ligands.sort()
Proteins = ["Example_Structures/"+x for x in listdir("Example_Structures/") if x[-3:] == "pdb"]
Proteins.sort()
Names = [x[19:23] for x in Ligands]

In [3]:
# Compute descriptors
ECIF = [GetECIF(protein, ligand, distance_cutoff=6.0) for protein, ligand in zip(Proteins, Ligands)]
ligand_descriptors = [GetRDKitDescriptors(x) for x in Ligands]

RDKit ERROR: [17:48:22] Explicit valence for atom # 17 C, 6, is greater than permitted
Traceback (most recent call last):
  File "/home/norberto/Installed/miniconda3/envs/rdkit/lib/python3.6/site-packages/rdkit/ML/Descriptors/MoleculeDescriptors.py", line 88, in CalcDescriptors
    res[i] = fn(mol)
  File "/home/norberto/Installed/miniconda3/envs/rdkit/lib/python3.6/site-packages/rdkit/Chem/QED.py", line 286, in qed
    qedProperties = properties(mol)
  File "/home/norberto/Installed/miniconda3/envs/rdkit/lib/python3.6/site-packages/rdkit/Chem/QED.py", line 249, in properties
    mol = Chem.RemoveHs(mol)
rdkit.Chem.rdchem.AtomValenceException: Explicit valence for atom # 17 C, 6, is greater than permitted
RDKit ERROR: [17:48:22] Explicit valence for atom # 63 C, 6, is greater than permitted
Traceback (most recent call last):
  File "/home/norberto/Installed/miniconda3/envs/rdkit/lib/python3.6/site-packages/rdkit/ML/Descriptors/MoleculeDescriptors.py", line 88, in CalcDescriptors
    re

In [4]:
# Merge ECIF with ligand descriptors
Descriptors = pd.DataFrame(ECIF, columns=PossibleECIF).join(pd.DataFrame(ligand_descriptors, columns=LigandDescriptors))
Descriptors.head()

,C;4;1;3;0;0-Br;1;1;0;0;0,C;4;1;3;0;0-C;3;3;0;1;1,C;4;1;3;0;0-C;4;1;1;0;0,C;4;1;3;0;0-C;4;1;2;0;0,C;4;1;3;0;0-C;4;1;3;0;0,C;4;1;3;0;0-C;4;2;0;0;0,C;4;1;3;0;0-C;4;2;1;0;0,C;4;1;3;0;0-C;4;2;1;0;1,C;4;1;3;0;0-C;4;2;1;1;1,C;4;1;3;0;0-C;4;2;2;0;0,...,fr_quatN,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_urea
0,0,0,0,0,3,0,0,0,15,8,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,3,0,0,0,2,6,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,2,2,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,5,0,0,1,22,10,...,1,0,0,0,0,0,0,0,0,0
4,0,0,0,0,14,0,0,10,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
# Load the previously saved model
model = pickle.load(open("ECIF6_LD_GBT.pkl", 'rb'))

In [6]:
# Make predictions
Results = pd.DataFrame(Names, columns=["ID"]).join(pd.DataFrame(model.predict(Descriptors), columns=["Predicted pK"]))
Results

,ID,Predicted pK
0,1a0q,7.428500
1,1a1b,6.283252
2,1a1c,6.372437
3,1a3e,8.256574
4,1a4h,5.963739
